# VGG16 Transfer Learning Model (LOCAL)

In [68]:
# Required libraries:

import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory


## 1. Split and Load Data

In [ ]:
# Install splitfolders library (in order to apply "train-val-test-split" to our data):

! pip install split-folders


In [ ]:
import splitfolders

# CHECK THESE DIRECTORIES BEFORE RUNNING CELL!

raw_data_path = "../raw_data/flowers"

split_data_path =  "../raw_data/split_folder"

# Split: 80% train, 10% val and 10% test:

splitfolders.ratio(raw_data_path, output=split_data_path, seed=42, ratio=(.8, .1, .1))


In [ ]:
# Load the train, validation, and test data:

train_data_dir = split_data_path+"/train/" # CHECK THIS DIRECTORY!

train_ds = image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    image_size=(128,128),
    batch_size=32,
    seed=42)

val_data_dir = split_data_path+"/val/" # CHECK THIS DIRECTORY!

val_ds = image_dataset_from_directory(
    val_data_dir,
    labels="inferred",
    label_mode="categorical",
    image_size=(128,128),
    batch_size=32,
    seed=42)

test_data_dir = split_data_path+"/test/" # CHECK THIS DIRECTORY!

test_ds = image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    image_size=(128,128),
    batch_size=32,
    seed=42)


## 2. Build CNN

In [61]:
# Create data augmentation layers:

data_augmentation = models.Sequential()

data_augmentation.add(layers.RandomFlip("horizontal"))
data_augmentation.add(layers.RandomZoom((0.1, 0.2)))
data_augmentation.add(layers.RandomTranslation(0.2, 0.2))
data_augmentation.add(layers.RandomRotation(0.1))

# Load VGG16 model from tensorflow:

vgg16_model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))

# Make sure we don't re-train the parameters:

vgg16_model.trainable = False

# Create the final layers, specific to our task:

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_1 = layers.Dropout(0.3)
dense_layer_2 = layers.Dense(256, activation='relu')
dropout_2 = layers.Dropout(0.4)
prediction_layer = layers.Dense(16, activation='softmax')

# Create full model architecture suited to our task:

inputs = layers.Input(shape = (128, 128, 3))

x = data_augmentation(inputs) # We still have our data augmentation layers

x = preprocess_input(x) # Then a preprocessing layer specifically designed for the VGG16

x = vgg16_model(x) # Then our transfer learning model

x = layers.Flatten()(x) # Flatten image tensors

x = layers.Dense(512, activation = "relu")(x) # Add a dense layer with dropout
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation = "relu")(x) # Add another dense layer with dropout
x = layers.Dropout(0.4)(x)

pred = layers.Dense(16, activation = "softmax")(x) # Add the prediction layer for 16-class classification

# Use the keras functional API to create our model:

final_model = models.Model(inputs = inputs, outputs = pred)

# Compile the model:

final_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [67]:
### WARNING: DO NOT RUN LOCALLY - VERY SLOW!!! ###
##################################################

# Fit the model:

es = EarlyStopping(patience=10,
                   restore_best_weights=True,
                   monitor='val_loss')

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor=0.1,
                       patience=3,
                       min_lr=0)

mcp = ModelCheckpoint("vgg16_tl_model.h5",
                      save_weights_only=False,
                      monitor='val_loss',
                      save_best_only=True)

history = final_model.fit(train_ds, epochs=200, validation_data=val_ds, callbacks=[es,lr,mcp], verbose=1)


## 3. Plot History (Loss & Accuracy)

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a subplot with 1 row and 2 columns
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the accuracy graph on the first subplot
ax1.plot(train_acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.legend(loc='best')
ax1.set_title('Training and Validation Accuracy')

# Plot the loss graph on the second subplot
ax2.plot(train_loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.legend(loc='best')
ax2.set_title('Training and Validation Loss')

# Display the graphs
plt.show()
